In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from tensorflow.keras.utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import LeakyReLU
from keras.models import Model
import keras
from sklearn.metrics import classification_report
import tensorflow as tf

import matplotlib.pyplot as plt
from keras.preprocessing import image
import cv2

In [ ]:
mnist_model = keras.models.load_model('/content/drive/MyDrive/midas/model_check/training0_9/new_new_50.hdf5')

From the link provided, the standard MNIST split is considered 16.67% for test data (10000 images as test and 60000 images as train)

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

Ideally we would want to change the input shape of the given model to 28x28 as we are dealing with MNIST data, but instead of having to go through the tedious task of having to resize the model's input layer, I shall resize the mnist data to be of 100x100 which is what the model was trained initially trained on.

In [ ]:
def extract_features(image_data=None):
  data = []
  for i in image_data:
    x = cv2.resize(i, dsize=(100, 100), interpolation=cv2.INTER_CUBIC)
    x = np.expand_dims(x, axis=2)
    x = np.array(x, dtype='float64')
    data.append(x)
  return np.asarray(data)
    
  
resized_x_train = extract_features(image_data=x_train)
resized_x_test = extract_features(image_data=x_test)
resized_y_train = tf.keras.utils.to_categorical(y_train, num_classes=10, dtype='float32')
resized_y_test = tf.keras.utils.to_categorical(y_test, num_classes=10, dtype='float32')

Evaluating MNIST data with previously trained data.
Despite the accuracy being lowm it does do a decent job of predicting. 
This does bring the accuracy up to 55% but still not ideal

In [ ]:
y_pred = np.argmax(mnist_model.predict(resized_x_test), axis=1)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.09      0.16       980
           1       0.96      0.12      0.21      1135
           2       0.40      0.84      0.54      1032
           3       0.56      0.83      0.67      1010
           4       0.85      0.58      0.69       982
           5       0.67      0.56      0.61       892
           6       0.64      0.90      0.75       958
           7       0.62      0.65      0.63      1028
           8       0.36      0.67      0.47       974
           9       0.79      0.33      0.47      1009

    accuracy                           0.55     10000
   macro avg       0.66      0.56      0.52     10000
weighted avg       0.67      0.55      0.52     10000



In [ ]:
optimizer = keras.optimizers.RMSprop(learning_rate=0.0001)
mnist_model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="/content/drive/MyDrive/midas/model_check/mnist_trained_on_previous_new.hdf5",
                                                 save_best_only=True,
                                                 verbose=1)

Training 1 epoch on the previously trained model we get the convergence time to be around 5 epochs from where the loss seems to plateau ar 0.03. The data is clearly being overfit at this point in time, but testing is required to be done on the 10000 images that were mentioned.

The accuracy obtained from the first epoch itself is around 90% which would lead to a much smaller convergence time.

In [ ]:
mnist_model.fit(resized_x_train, resized_y_train, verbose = 1, batch_size=32, callbacks=cp_callback, validation_data=(resized_x_test, resized_y_test), epochs=10)

Epoch 1/10
1875/1875 [==============================] - 35s 18ms/step - loss: 0.3347 - accuracy: 0.9003 - val_loss: 0.0499 - val_accuracy: 0.9864

Epoch 00001: val_loss improved from inf to 0.04992, saving model to /content/drive/MyDrive/midas/model_check/mnist_trained_on_previous_new.hdf5
Epoch 2/10
1875/1875 [==============================] - 33s 17ms/step - loss: 0.0876 - accuracy: 0.9789 - val_loss: 0.0342 - val_accuracy: 0.9903

Epoch 00002: val_loss improved from 0.04992 to 0.03424, saving model to /content/drive/MyDrive/midas/model_check/mnist_trained_on_previous_new.hdf5
Epoch 3/10
1875/1875 [==============================] - 33s 17ms/step - loss: 0.0577 - accuracy: 0.9851 - val_loss: 0.0339 - val_accuracy: 0.9902

Epoch 00003: val_loss improved from 0.03424 to 0.03386, saving model to /content/drive/MyDrive/midas/model_check/mnist_trained_on_previous_new.hdf5
Epoch 4/10
1875/1875 [==============================] - 33s 18ms/step - loss: 0.0457 - accuracy: 0.9889 - val_loss: 0.0

Evaluating the previously trained model (0-9) after training on 1 epoch with the mnist data.

In [ ]:
y_pred = np.argmax(mnist_model.predict(resized_x_test), axis=1)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       980
           1       0.99      1.00      1.00      1135
           2       1.00      0.98      0.99      1032
           3       1.00      0.99      0.99      1010
           4       0.99      1.00      0.99       982
           5       0.99      0.99      0.99       892
           6       1.00      0.98      0.99       958
           7       0.99      0.99      0.99      1028
           8       0.98      0.99      0.99       974
           9       0.99      0.99      0.99      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000



In [ ]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu', input_shape = (100, 100, 1)))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same'))
model.add(LeakyReLU(alpha=0.1))

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same'))
model.add(LeakyReLU(alpha=0.3))

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same'))
model.add(LeakyReLU(alpha=0.3))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same'))
model.add(LeakyReLU(alpha=0.1))

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same'))
model.add(LeakyReLU(alpha=0.3))

model.add(MaxPool2D(pool_size=(2,2)))


model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same'))
model.add(LeakyReLU(alpha=0.1))

model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 512, kernel_size = (3,3),padding = 'Same'))
model.add(LeakyReLU(alpha=0.3))

model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same'))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.3))
model.add(Dense(128, activation = "relu"))
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.25))
model.add(Dense(10, activation = "softmax"))  

Evaluating the performace agter random initialization. We see the accuracy is much lower in comparison.

In [ ]:
y_pred = np.argmax(model.predict(resized_x_test), axis=1)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       980
           1       0.00      0.00      0.00      1135
           2       0.12      0.01      0.01      1032
           3       0.00      0.00      0.00      1010
           4       0.10      1.00      0.18       982
           5       0.00      0.00      0.00       892
           6       0.00      0.00      0.00       958
           7       0.00      0.00      0.00      1028
           8       0.00      0.00      0.00       974
           9       0.00      0.00      0.00      1009

    accuracy                           0.10     10000
   macro avg       0.02      0.10      0.02     10000
weighted avg       0.02      0.10      0.02     10000



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training from a random model to measure the convergence.
We notice that here, the accuracy starts from a much lower & and converges much later.

In [ ]:
optimizer = keras.optimizers.RMSprop(learning_rate=0.0001)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

model.fit(resized_x_train, resized_y_train, validation_data=(resized_x_test, resized_y_test), verbose = 1, batch_size=32, callbacks=None, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 35s 18ms/step - loss: 1.0560 - accuracy: 0.6306 - val_loss: 0.0527 - val_accuracy: 0.9851
Epoch 2/10
1875/1875 [==============================] - 33s 17ms/step - loss: 0.1106 - accuracy: 0.9723 - val_loss: 0.0396 - val_accuracy: 0.9893
Epoch 3/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0675 - accuracy: 0.9836 - val_loss: 0.0313 - val_accuracy: 0.9904
Epoch 4/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0484 - accuracy: 0.9882 - val_loss: 0.0309 - val_accuracy: 0.9926
Epoch 5/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0449 - accuracy: 0.9897 - val_loss: 0.0340 - val_accuracy: 0.9922
Epoch 6/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0430 - accuracy: 0.9906 - val_loss: 0.0335 - val_accuracy: 0.9925
Epoch 7/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0396 - accuracy: 0.9912 - val_loss: 0.0343 -

In [ ]:
y_pred = np.argmax(model.predict(resized_x_test), axis=1)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       980
           1       1.00      0.99      0.99      1135
           2       0.95      1.00      0.97      1032
           3       0.97      1.00      0.98      1010
           4       0.99      0.99      0.99       982
           5       1.00      0.96      0.98       892
           6       1.00      0.97      0.99       958
           7       0.99      0.99      0.99      1028
           8       0.99      0.98      0.99       974
           9       0.99      0.99      0.99      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000

